# PART1

## Json import to openSearch

In [ ]:
import createRecipesMap as crm

crm.CreateRecipesMap("../jsonData/recipe_queries_results.json", '../jsonData/recipesMap.json').createMap()

## Login to openSearch

In [ ]:
import loginOpenSearch as lop

login = lop.LoginOpenSearch("../jsonData/openSearchConfig.json").login()
    
client = login[0]
index_name = login[1]

## Create index structure

In [ ]:
import createIndexes as ci

ci.CreateIndexes(client, index_name).createIndexStructure()

## Choose the tokenizer and model that will be used in the embeddings

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")
model = AutoModel.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")

In [ ]:
import computeEmbeddings as ce

ce.ComputeEmbeddings("../jsonData/recipesMap.json","../jsonData/recipesEmbeddingsMap.json", tokenizer, model).createMap()

In [ ]:
import indexRecipes as ir

ir.IndexRecipes("../jsonData/recipesMap.json", "../jsonData/recipesEmbeddingsMap.json", client, index_name).indexRecipes()


In [ ]:
import search as s

s.Search(client, index_name, tokenizer, model).queryOpenSearch('Holiday Salad', 5,None, None, ["salads"], ["lupine"], None)

## Testing

In [ ]:
import searchJson as sj
metrics = sj.SearchJson(client, index_name, tokenizer, model)
metrics.searchJson("../jsonData/cleanAnnotations.json")

In [ ]:
delete this sentence to run
if client.indices.exists(index=index_name):
    # Delete the index.
    response = client.indices.delete(
        index = index_name,
        timeout = "600s"
    )
    print('\nDeleting index:')
    print(response)

# PART2

## Compute the step similarity matrix

### Compute the steps json files

In [ ]:
import stepsCalculator as sc

sc.CreateStepsMaps("../jsonData/recipesMap.json", '../jsonData/stepsImage.json', '../jsonData/stepsNoImage.json').createMaps()


### Compute the embeddings

In [ ]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"

device

In [ ]:
import computeImagesEmbeddings as cie

cie.ComputeImagesEmbeddings("../jsonData/stepsImage.json","../jsonData/stepsNoImage.json","../jsonData/stepsImageEmbeddingsMap.json","../jsonData/stepsNoImageEmbeddingsMap.json","images/").createMap()

### Compute the similarity matrix

In [ ]:
import computeMatrix as cm

imageSimilarityMatrix = cm.ComputeMatrix("../jsonData/stepsImageEmbeddingsMap.json", "../jsonData/stepsNoImageEmbeddingsMap.json").createMatrix()
print(imageSimilarityMatrix)
print(len(imageSimilarityMatrix[0]))

In [ ]:
import getImage as gi

gi.GetImage(imageSimilarityMatrix, "../jsonData/stepsImage.json", "../jsonData/stepsNoImage.json", "images/","../jsonData/recipesMap.json").getImageFunc()

In [ ]:
#import computeClipOutput as cco

#image = cco.ComputeClipOutput(imageSimilarityMatrix, "../jsonData/stepsImage.json", "../jsonData/stepsNoImage.json", "images/","../jsonData/recipesMap.json").getImage()

# PART3


In [1]:
import os
import numpy as np
import transformers
import json
import loginOpenSearch as lop
import search as s
import extractiveQA as eq

import random

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AutoModel,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)

model_finetuned = '/user/data/public/twiz-intent-model'
with open(os.path.join(model_finetuned + '/all_intents.json'), 'r') as all_intents_json:
    all_intents = json.load(all_intents_json) # contains the written out names of intents. also implicitly

tokenizer_name = 'roberta-base' # try 'bert-base-uncased', 'bert-base-cased', 'bert-large-uncased'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name) # loads a tokenizer

model = AutoModelForSequenceClassification.from_pretrained(model_finetuned, num_labels=len(all_intents)) # Loads the BERT model weights

login = lop.LoginOpenSearch("../jsonData/openSearchConfig.json").login()
openSearchTokenizer = AutoTokenizer.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")
openSearchModel = AutoModel.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")

searchEngine = s.Search(login[0], login[1], openSearchTokenizer, openSearchModel)
qaExtractor = eq.ExtractiveQA()



----------------------------------------------------------------------------------- INDEX SETTINGS
{'user209': {'settings': {'index': {'creation_date': '1649609449190',
                                    'knn': 'true',
                                    'number_of_replicas': '0',
                                    'number_of_shards': '4',
                                    'provided_name': 'user209',
                                    'refresh_interval': '1s',
                                    'uuid': 'zYvHS7n3SS6ejuPIp7Ikhw',
                                    'version': {'created': '135238227'}}}}}

----------------------------------------------------------------------------------- INDEX MAPPINGS
{'user209': {'mappings': {'properties': {'description': {'similarity': 'BM25',
                                                         'type': 'text'},
                                         'ingredients': {'type': 'keyword'},
                                         'negative_Ke

In [2]:
import stateMachine
turing = stateMachine.StateMachine(tokenizer,model, all_intents,searchEngine, qaExtractor)
       
        #greeting
response = 'hello, today i would like to cook pizza.'
intent = turing.getIntent(response)
print(intent)
turing.setUserResponse(response)
getattr(turing, intent)()

        #ask_for_desired_ingredients
response = 'i would like cheese please'
intent = turing.getIntent(response)
print(intent)
turing.setUserResponse(response)
getattr(turing, intent)()

        #ask_for_unwanted_ingredients
response = 'i really dont want bananas or olives'
intent = turing.getIntent(response)
print(intent)
turing.setUserResponse(response)
getattr(turing, intent)()

        #ask_for_keywords
response = 'yes, i would like it to be vegan please' 
print(intent)
intent = turing.getIntent(response)
turing.setUserResponse(response)
getattr(turing, intent)()

        #ask_for_time_restrictions
response = 'im in a hurry so thirty minutes in the maximum please' 
#response = 'yes i would like the recipe to be under sixty minutes' 
intent = turing.getIntent(response)
print(intent)
turing.setUserResponse(response)
getattr(turing, intent)()

        #show_top_recipes
response = 'i want to see the first recipe thank you' 
intent = turing.getIntent(response)
print(intent)
turing.setUserResponse(response)
getattr(turing, intent)()

        #show_ingredients / skip ings
response = 'yes' 
intent = turing.getIntent(response)
print(intent)
turing.setUserResponse(response)
getattr(turing, intent)()

        #show_steps
while turing.state != 'end' :
    #response = input("USER:")
    response = 'yes' 
    intent = turing.getIntent(response)
    print(intent)
    turing.setUserResponse(response)
    getattr(turing, intent)()
#steps



Hello! I'm a cooking assistant and I'm here to help you cook anything you want. What would you like to prepare today?
IdentifyProcessIntent
Ok! I'll now ask you some questions to find the pizza recipe you're looking for!
Are there any desired ingredients you'd like the recipe to have? If so, could you enumerate them?
IdentifyProcessIntent
Ok, I'll take cheese in consideration.
Are there any ingredients you really don't want in the recipe? If so, could you also enumerate them?
QuestionIntent
Got it, I'll find recipes which don't contain any bananas and olives.
Should the recipe follow any dietary requirements?
For example, does it need to be vegan or gluten-free?
QuestionIntent
Ok, I'll prioritize recipes that have vegan requirements.
Last question! Are there any time restrictions for the food preparation? If so, how many minutes would you like it to take?
AMAZON.YesIntent
Noted, I'll try to look for recipes that take less then 30 minutes.
Done! I have found some recipes that fit your d

Which one would you like to see?
AMAZON.SelectIntent
You picked Home-Made Pizza. Great choice!
Would you like to know the needed ingredients?
AMAZON.YesIntent
For this recipe you'll need the following ingredients:


Are you ready to start cooking?
AMAZON.YesIntent
Ok! Let's begin!



Would you like to proceed to the next step?
AMAZON.YesIntent


Would you like to proceed to the next step?
AMAZON.YesIntent


...

Would you like to proceed to the next step?
AMAZON.YesIntent


Would you like to proceed to the next step?
AMAZON.YesIntent


Would you like to proceed to the next step?
AMAZON.YesIntent


Would you like to proceed to the next step?
AMAZON.YesIntent


This was the last step, would you like to proceed?
AMAZON.YesIntent
Now that you've finished cooking you can finally enjoy your meal! Bon appétit!


In [ ]:
import stateMachine
turing = stateMachine.StateMachine(tokenizer,model, all_intents, searchEngine, qaExtractor)
while(turing.state != 'end'):

    response = input("USER:")
    intent = turing.getIntent(response)
    print(intent)
    turing.setUserResponse(response)
    try: 
        getattr(turing, intent)() 
    except Exception as e:
        print ('I\'m sorry, I couldn\'t quite understand you! Could you please rephrase your answer?')
        continue

        